In [28]:
from eth_account import Account
import requests
import json
import random

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [29]:
import psycopg2

In [30]:
from config import config, cross, nftport, simplehash, dinary

In [5]:
def insert_batch(name, description):
    """
    Save the name and description for each collection
    Arguments:
        name: collection name
        description: short description for each nft or collection
    """
    sql = "INSERT INTO batchs (name, description, images, attributes) VALUES(%s, %s, '{}', '{}') returning id_batchs;"
    conn = None
    id_of_new_row = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        cur.execute(sql, (name, description))
        id_of_new_row = cur.fetchone()[0]
        conn.commit()
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            
    return id_of_new_row

In [6]:
# insert_batch(name = "Monkey", description = "Bananas!")

In [7]:
def insert_emails_list(email_list):
    data_email_list = []
    for email in email_list:
        acct = Account.create(email)
        data_email_list.append((email, acct.privateKey, acct.address))
    sql = "INSERT INTO emails(email, walletpk, wallet) VALUES(%s, %s, %s) ON CONFLICT DO NOTHING"
    conn = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        cur.executemany(sql, data_email_list)
        conn.commit()
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return True

In [8]:
# insert_emails_list(['miki@clinica.top', 'frankmosheh@gmail.com', 'mateus@clinica.top'])

In [36]:
def insert_images(images, id_batchs):
    sql = "UPDATE batchs SET images = %s WHERE id_batchs = %s"
    conn = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        cur.execute(sql, (images, id_batchs))
        conn.commit()
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return True

In [37]:
# id_batchs = 6
# images = json.dumps({"images":["hole.com","msn.com"]})
# insert_images(images, id_batchs)

True

In [ ]:
def insert_sent_list(sent_list):
    sql = "INSERT INTO sent(id_batchs, email) VALUES(%s, %s) ON CONFLICT DO NOTHING"
    conn = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        cur.executemany(sql, sent_list)
        conn.commit()
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return True

In [53]:
def update_sent_list(sent_list):
    sql = "UPDATE sent SET tx_hash = %s WHERE id_batchs = %s AND email = %s"
    conn = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        cur.executemany(sql, sent_list)
        conn.commit()
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return True

In [10]:
# update_sent_list([
#     ('hash','miki@clinica.top','3'),
#     ('hash','frankmosheh@gmail.com','3'),
#     ('hash','mateus@clinica.top','3'),
# ])

In [62]:
def get_emails_to_send(id_batchs):
    """
    A dataframe from aggregated_investments table in order to save a snapshot
    Returns:
        snapshot: a dataframe with all the relevan fields to save
    """
    sql = "SELECT emails.email, emails.wallet FROM sent, emails WHERE sent.id_batchs = %s AND sent.email = emails.email AND sent.tx_hash IS NULL"
    conn = None
    data = []
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        cur.execute(sql, (id_batchs,))
        email_users = cur.fetchall()
        for row in email_users:
            data.append({'email': row[0], 'wallet': row[1]})
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    
    return data

In [63]:
# get_emails_to_send(id_batchs = 8)

In [64]:
def get_batch_metadata(id_batchs):
    """
    A dataframe from aggregated_investments table in order to save a snapshot
    Returns:
        snapshot: a dataframe with all the relevan fields to save
    """
    sql = "SELECT name, description, images FROM batchs WHERE id_batchs = %s"
    conn = None
    data = []
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        cur.execute(sql, (id_batchs,))
        batch_data = cur.fetchone()
        data = {'name': batch_data[0], 'description': batch_data[1], 'images': batch_data[2]}
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    
    return data

In [65]:
# get_batch_metadata(id_batchs = 3)

In [77]:
def send_nft(address, name, description, image_url):
    nftport_params = nftport()
    headers={"Authorization": nftport_params['nftport_secret'], "Content-Type": "application/json"}
    payload = {
        "chain": "polygon",
        "name": name,
        "description": description,
        "file_url": image_url,
        "mint_to_address": address
    }
    response = requests.post('https://api.nftport.xyz/v0/mints/easy/urls', json=payload, headers=headers)
    return response.json()

In [78]:
def send_batch_nfts(id_batchs):
    emails_data = get_emails_to_send(id_batchs = id_batchs)
    batch_data = get_batch_metadata(id_batchs = id_batchs)
    sent_data = []
    for user in emails_data:
        metadata = {
            'name': batch_data['name'],
            'image': random.choice(batch_data['images']['images']),
            'description': batch_data['description'],
        }
        response = send_nft(address=user['wallet'], name=batch_data['name'], description=batch_data['description'], image_url=random.choice(batch_data['images']['images']))
        sent_data.append((response, user['email'], id_batchs))
    return sent_data

In [79]:
def deploy_nfts(id_batchs):
    sent_data = send_batch_nfts(id_batchs)
    if len(sent_data) > 0:
        filtered_sent_data = [(x[0]['transaction_hash'], x[2], x[1]) for x in sent_data]
        update_sent_list(sent_list = filtered_sent_data)

In [80]:
# deploy_nfts(id_batchs = 8)

In [75]:
def get_emails_and_txhash(id_batchs):
    """
    A dataframe from aggregated_investments table in order to save a snapshot
    Returns:
        snapshot: a dataframe with all the relevan fields to save
    """
    sql = "SELECT emails.email, emails.wallet, sent.tx_hash FROM sent, emails WHERE sent.id_batchs = %s AND sent.email = emails.email AND sent.tx_hash IS NOT NULL"
    conn = None
    data = []
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        cur.execute(sql, (id_batchs,))
        email_users = cur.fetchall()
        for row in email_users:
            data.append({'email': row[0], 'wallet': row[1], 'txid': row[2]})
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    
    return data

In [76]:
get_emails_and_txhash(id_batchs = 8)

[{'email': 'maria@clinica.top',
  'wallet': '0xD1fE4BeE31e2ca8aFD5DED4cacf5F851786f8FB0',
  'txid': '0xa1622f17c509513a69a68db473877081d4b92ed24f87bf1988f214dbc2d2bce5'},
 {'email': 'frank@clinica.top',
  'wallet': '0x5728335CFfD8fa47B448E3254Ea45EBF4d9a72C7',
  'txid': '0xb3a23f33da51751b67109b2f9eb06d6f9e9ddbe4fd9fe2a968a6060be8647807'}]

---

In [87]:
def get_wallet_by_email(email):
    """
    A dataframe from aggregated_investments table in order to save a snapshot
    Returns:
        snapshot: a dataframe with all the relevan fields to save
    """
    sql = "SELECT emails.wallet FROM emails WHERE emails.email = %s"
    conn = None
    email_users = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        cur.execute(sql, (email,))
        email_users = cur.fetchone()[0]
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    
    return email_users

In [88]:
def get_nfts_by_wallet(wallet):
    simplehash_params = simplehash()
    url = "https://api.simplehash.com/api/v0/nfts/owners?chains=polygon&wallet_addresses=" + wallet
    headers = {
        "accept": "application/json",
        "X-API-KEY": simplehash_params['simplehash_secret']
    }

    response = requests.get(url, headers=headers)
    return (wallet, response.json()['nfts'])

In [91]:
def get_nfts(email):
    data_wallet = get_wallet_by_email(email)
    if data_wallet:
        data = get_nfts_by_wallet(wallet = data_wallet)
        return data
    else:
        return None, None

In [94]:
get_nfts(email = "frankmosheh@gmail.com")

('0x54707B62701620c8401DCBDCD2e579Ea0231cD8D',
 [{'nft_id': 'polygon.0x55a8dbe6f191b370885d01e30cb7d36d0fa99f16.6287',
   'chain': 'polygon',
   'contract_address': '0x55A8DBe6F191B370885D01E30CB7D36d0fA99F16',
   'token_id': '6287',
   'name': 'BBVA 2022',
   'description': 'La mejor hackathon!',
   'image_url': 'https://cdn.simplehash.com/assets/1a21ff6a67a812c0bce96949d6dcd3ecf8c597ef085ce8759296a4d1577330f6.png',
   'video_url': None,
   'audio_url': None,
   'model_url': None,
   'previews': {'image_small_url': 'https://lh3.googleusercontent.com/4RanB1j38xsWQwLVAvrSP8d0RFLWXueYm9cqEJ5i_-kT7w_-cMyd5gC2KgrDOaenHlz9JQA-CUXry9TaEL70sEjAZlvBfexv8yQ5=s250',
    'image_medium_url': 'https://lh3.googleusercontent.com/4RanB1j38xsWQwLVAvrSP8d0RFLWXueYm9cqEJ5i_-kT7w_-cMyd5gC2KgrDOaenHlz9JQA-CUXry9TaEL70sEjAZlvBfexv8yQ5',
    'image_large_url': 'https://lh3.googleusercontent.com/4RanB1j38xsWQwLVAvrSP8d0RFLWXueYm9cqEJ5i_-kT7w_-cMyd5gC2KgrDOaenHlz9JQA-CUXry9TaEL70sEjAZlvBfexv8yQ5=s1000',
    'i

---

In [1]:
import cloudinary
import cloudinary.uploader
import cloudinary.api

In [19]:
import json

In [10]:
cloudinary_config = cloudinary.config(cloud_name='climatempo', api_secret='WwH6HWQ0ALwhlrphu4zD9RRQ37I', api_key='689443847436934')

****1. Set up and configure the SDK:****
Credentials:  climatempo WwH6HWQ0ALwhlrphu4zD9RRQ37I 



In [13]:
def uploadImage():
    cloudinary.uploader.upload("bbva.jpg", public_id="bbva2022")
    srcURL = cloudinary.CloudinaryImage("bbva2022").build_url()
    print("****2. Upload an image****\nDelivery URL: ", srcURL, "\n")

In [23]:
uploadImage()

****2. Upload an image****
Delivery URL:  https://res.cloudinary.com/climatempo/image/upload/bbva2022 



In [44]:
import uuid

In [45]:
str(uuid.uuid4())

'1664b646-423e-47cc-a0ed-243bb520f80b'